In [ ]:
import streamlit as st
import cv2
import numpy as np
import tempfile
import pandas as pd

# Load CSVs
resultmen = pd.read_csv('codebase/resultmen.csv')
resultwomen = pd.read_csv('codebase/resultwomen.csv')

# --- Helper: Get suggestions from CSV ---
def get_suggestions(gender, face_shape, interocular_category, nose_category):
    if gender == "Man":
        df = resultmen
        beard_col = 'beard_suggestions'
    else:
        df = resultwomen
        beard_col = None
    row = df[
        (df['face_shape'] == face_shape) &
        (df['interocular_category'] == interocular_category) &
        (df['nose_category'] == nose_category)
    ]
    if not row.empty:
        hair = row.iloc[0]['hairstyle_suggestions']
        beard = row.iloc[0][beard_col] if beard_col else None
        return hair, beard
    else:
        return "No specific suggestions found.", None

# --- Sidebar: User choice ---
st.sidebar.title("Face Shape Predictor")
gender = st.sidebar.radio("Select Gender", ["Man", "Woman"])

st.title("Real-Time Face Shape & Hairstyle Suggestion")
st.write("Please allow camera access and click 'Capture'.")

# --- Camera access ---
img_file_buffer = st.camera_input("Take a picture")

if img_file_buffer is not None:
    # Save image to temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temp_file:
        temp_file.write(img_file_buffer.getvalue())
        image_path = temp_file.name
    st.image(image_path, caption="Captured Image", use_column_width=True)

    # --- Run Model ---
    if gender == "Man":
        from codebase.men import predict_face_shape as men_predict
        model_result = men_predict(image_path)
        result_csv = resultmen
    else:
        from codebase.women import predict_face_shape as women_predict
        model_result = women_predict(image_path)
        result_csv = resultwomen

    # --- Run Metrics ---
    from codebase.metrics import run_metrics
    metrics_result = run_metrics(image_path)

    # --- Compare Results ---
    if model_result['face_shape'] == metrics_result['face_shape']:
        final_result = model_result
        st.success("Model and metrics agree on face shape!")
    else:
        final_result = metrics_result
        st.warning("Model and metrics disagree. Using metrics result.")

    # --- Show Results ---
    st.subheader("Predicted Face Shape")
    st.write(final_result['face_shape'])

    st.subheader("Facial Measurements")
    st.write(f"Interocular: {final_result['interocular_category']}")
    st.write(f"Nose Width: {final_result['nose_category']}")

    # --- Hairstyle Suggestions ---
    hair, beard = get_suggestions(gender, final_result['face_shape'], final_result['interocular_category'], final_result['nose_category'])
    st.subheader("Hairstyle Suggestions")
    if hair:
        for s in hair.split(';'):
            st.markdown(f"- {s.strip()}")
    else:
        st.info("No specific suggestions found. Try another image.")

    # For men, show beard suggestions
    if gender == "Man" and beard:
        st.subheader("Beard Suggestions")
        for s in beard.split(';'):
            st.markdown(f"- {s.strip()}")

# --- UI Styling ---
st.markdown("""
<style>
    .stApp {background-color: #f5f5fa;}
    .stSidebar {background-color: #e0e0ef;}
    .stSuccess {color: green;}
    .stWarning {color: orange;}
</style>
""", unsafe_allow_html=True)
